Lab02-2
============

###  Context
#### Classification
   + Logistic Regression



# Logistic Regression

Logistic Regression은 클래스 분류문제를 해결하기 위해 사용되는 것으로 독립 변수의 선형 결합으로 종속 변수를 설명하는 부분은 앞서 배운 선형 회귀(Linear Regression)와 같으나 그 결과가 특정 클래스일 확률로 나와 클래스 분류문제를 해결할 수 있는 방법입니다.

## odds
어떤 일이 발생할 상대적인 비율
발생할 확률 p 와 발생하지 않을 확률 q(= 1- p)
두 확률의 상대적인 비율

![title](Images/ODD_F.png)

## Log(odds)

1. 0을 기준으로 상호 대칭적
2. 계산이 용이

![title](Images/ODD_G.png)

## Logistic Function(sigmoid)
해당 클래스일 확률 Y를 Log(odds)로 보고 이를 유도해보자

![title](Images/sigmoid_F1.png)

![title](Images/sigmoid_F2.png)

위에 정리된 식을 가설 그래프로 나타내면

![title](Images/sigmoid_G.png)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os.path import join
from matplotlib.colors import ListedColormap
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.model_selection import train_test_split
from scipy import optimize as op
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
from scipy import interp



# 1. 데이터 살펴보기
앞서 사용했던 붓꽃 데이터를 사용합니다.

독립 변수로는 꽃받침 길이, 꽃받침 너비, 꽃잎의 길이, 꽃잎의 너비 데이터가 종속 변수로는 붓꽃 종(Species)입니다.

In [ ]:
iris_analysis = pd.read_csv(join('data', 'Iris.csv'))

iris_analysis.head()

In [ ]:
iris_setosa = iris_analysis.loc[iris_analysis['Species'] == 'Iris-setosa']
iris_versicolor = iris_analysis.loc[iris_analysis['Species'] == 'Iris-versicolor']
iris_virginica = iris_analysis.loc[iris_analysis['Species'] == 'Iris-virginica']

In [ ]:
fig, axs = plt.subplots(2,2)

#꽃받침의 길이
axs[0, 0].boxplot([iris_setosa['SepalLengthCm'], iris_versicolor['SepalLengthCm'], iris_virginica['SepalLengthCm']],
           labels = ['seotosa', 'versicolor', 'virginica'],
           meanline = True,
           showmeans = True)
axs[0, 0].set_title('Sepal length Boxplot')

#꽃받침의 너비
axs[0, 1].boxplot([iris_setosa['SepalWidthCm'], iris_versicolor['SepalWidthCm'], iris_virginica['SepalWidthCm']],
           labels = ['seotosa', 'versicolor', 'virginica'],
           meanline = True,
           showmeans = True)
axs[0, 1].set_title('Sepal width Boxplot')

#꽃잎의 길이
axs[1, 0].boxplot([iris_setosa['PetalLengthCm'], iris_versicolor['PetalLengthCm'], iris_virginica['PetalLengthCm']],
           labels = ['seotosa', 'versicolor', 'virginica'],
           meanline = True,
           showmeans = True)
axs[1, 0].set_title('Petal length Boxplot')

#꽃잎의 너비
axs[1, 1].boxplot([iris_setosa['PetalWidthCm'], iris_versicolor['PetalWidthCm'], iris_virginica['PetalWidthCm']],
           labels = ['seotosa', 'versicolor', 'virginica'],
           meanline = True,
           showmeans = True)
axs[1, 1].set_title('Petal width Boxplot')

fig.subplots_adjust(left = 0.01, right = 2, bottom = 0.05, top = 2, hspace = 0.3, wspace = 0.3)

plt.show()

In [ ]:
iris = pd.read_csv(join('data', 'Iris.csv'))
iris.head()

# 2. 간단한 전처리
모든 독립 변수와 별개로 상수항을 추가하기 위해 ones함수를 이용하여 하나의 열을 추가하고 나머지 데이터를 다시 제자리에 저장합니다.

학습용 데이터와 테스트용 데이터를 7:3의 비율로 나눕니다.

In [ ]:
Species = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

#데이터의 수
m = iris.shape[0]

#독립 변수의 수
n = 4

#클래스 의 수
k = 3

X = np.ones((m,n + 1))
y = np.array((m,1))

X[:,1] = iris['PetalLengthCm'].values
X[:,2] = iris['PetalWidthCm'].values
X[:,3] = iris['SepalLengthCm'].values
X[:,4] = iris['SepalWidthCm'].values

y = iris['Species'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 11)

# 3. scratch로 분류 해보기

### sigmoid
위의 유도한 Logistic Function으로 z에 독립 변수와 계수의 스칼라곱을 수행하여 활용할 함수입니다.

In [ ]:
def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

### 비용함수
실제 값과 연산으로 추정된 값간의 차이를 연산해주는 함수로 이 비용함수가 최소값을 가지게 하는 계수들의 집합(theta)를 찾는 것이 Logistic Regression의 목표이다. 최소값을 찾기위해 경사하강법을 사용하는데 이론 수업에서 자세히 다루지 않으므로 존재만 확인하고 넘어가자

![title](Images/LRC_F.png)

![title](Images/LRG_F.png)

In [ ]:
def regCostFunction(theta, X, y, _lambda = 0.1):
    m = len(y)
    h = sigmoid(X.dot(theta))
    reg = (_lambda/(2 * m)) * np.sum(theta**2)

    return (1 / m) * (-y.T.dot(np.log(h)) - (1 - y).T.dot(np.log(1 - h))) + reg

def regGradient(theta, X, y, _lambda = 0.1):
    m, n = X.shape
    theta = theta.reshape((n, 1))
    y = y.reshape((m, 1))
    h = sigmoid(X.dot(theta))
    reg = _lambda * theta /m

    return ((1 / m) * X.T.dot(h - y)) + reg

### logisticRegression(X, y, theta)
위의 비용함수를 최소화한 결과를 scipy의 optimize.minimize를 이용하여 구하는 함수로 X에는 학습용 데이터, y에는 학습용 데이터의 답, theta에는 초기 계수들의 집합을 넣어 최적화된 계수들을 반환하는 함수입니다.

In [ ]:
def logisticRegression(X, y, theta):
    result = op.minimize(fun = regCostFunction, x0 = theta, args = (X, y),
                         method = 'TNC', jac = regGradient)
    
    return result.x

In [ ]:
#계수를 저장할 변수
all_theta = np.zeros((k, n + 1))

#학습 수행
i = 0
for flower in Species:
    #현재 구하고자하는 종은 1 아닌 값은 0
    tmp_y = np.array(y_train == flower, dtype = int)
    optTheta = logisticRegression(X_train, tmp_y, np.zeros((n + 1,1)))
    all_theta[i] = optTheta
    i += 1
    
print(all_theta)

In [ ]:
#추론부분
P = sigmoid(X_test.dot(all_theta.T)) #각 종에 대한 모든 데이터의 확률값
p = [Species[np.argmax(P[i, :])] for i in range(X_test.shape[0])] #추론된 확률 중 가장 높은 값을 채택하여 그 종으로 최종 추론

print("테스트 세트 정확도:", accuracy_score(y_test, p) * 100 , '%')

# 4. scratch로 분류한 결과 확인하기
행 값이 추론결과 열 값이 실제 값으로 이루어진 Confusion Matrix로 시각화 해보자

In [ ]:
cfm = confusion_matrix(y_test, p, labels = Species)

fig, ax = plt.subplots()
im = ax.imshow(cfm)

ax.set_xticks(np.arange(len(Species)))
ax.set_yticks(np.arange(len(Species)))

ax.set_xticklabels(Species)
ax.set_yticklabels(Species)

for i in range(len(Species)):
    for j in range(len(Species)):
        text = ax.text(j, i, cfm[i, j],
                       ha="center", va="center", color="r", fontsize=20)

plt.show()

# 5. sklearn으로 분류해보기

In [ ]:
iris = datasets.load_iris()
X = iris.data[:, 2:4]
Y = iris.target

logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')

logreg.fit(X, Y)

x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
h = .02
#cmap_light = Mash에 표시될 3가지 색상
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
#cmap_bold = 학습 데이터를 표시할 3가지 색상
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = logreg.predict(np.c_[xx.ravel(), yy.ravel()])

Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(4, 3))
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k', cmap=cmap_bold, s=15)
plt.xlabel('Petal length')
plt.ylabel('Petal width')

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())

plt.show()

# 6. sklearn 결과 평가하기

## 6-1 정확도

In [ ]:
iris = datasets.load_iris()
X = iris.data[:,:]
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=11)

logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial', max_iter = 1000)

logreg.fit(X_train, y_train)

In [ ]:
print("학습 세트 정확도: {:.3f}%".format(logreg.score(X_train, y_train) * 100))
print("테스트 세트 정확도: {:.3f}%".format(logreg.score(X_test, y_test) * 100))

In [ ]:
iris = datasets.load_iris()
X = iris.data[:,:]
y = iris.target

logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial', max_iter = 1000)

y = label_binarize(y, classes=[0, 1, 2])
n_classes = y.shape[1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=11)

In [ ]:
n_samples, n_features = X.shape

colors = (['aqua', 'darkorange', 'cornflowerblue'])

classifier = OneVsRestClassifier(logreg)
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
plt.figure()

for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()
